In [1]:
from functools import partial

from dotenv import load_dotenv; load_dotenv()
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

import librun
from libdatasets import *
from libadversarial import uncertainty_stop

In [2]:
def saturation(learner, metrics):
    "Finite differences saturation stop condition."
    
    # for testing
    return learner.y_training.shape[0] >= 1010
    
    accuracy = metrics.frame.accuracy_score
    diff = accuracy.iloc[-1]-accuracy.iloc[-2]
    # TODO: Decide on this condition, it might need to involve the passive scores and
    # accuracy per 1000 estimates from stop_saturation_estimation.ipynb
    if diff > 0 and diff < 1e-3:
        return True
    return False

In [3]:
matrix_A = {
    "datasets": [
        ("spamassassin", wrap(spamassassin, None)),
    ],
    "dataset_mutators": {
        "none": (lambda *x, **kwargs: x),
    },
    "methods": [
        ("uncertainty", partial(uncertainty_stop, n_instances=10)),
    ],
    "models": ["svm-linear"],
    "meta": {
        "dataset_size": 1000,
        "labelled_size": 10,
        "test_size": 0.5,
        "n_runs": 10,
        "ret_classifiers": True,
        "ensure_y": True,
        "stop_info": True,
        "aggregate": False,
        "stop_function": ("len1000", lambda learner: learner.y_training.shape[0] >= 1000),
        "pool_subsample": 1000,
    },
}
matrix_B = {
    **matrix_A,
    "meta": {
        **matrix_A["meta"],
        "stop_function": ("sat1", saturation)
    }
}

In [4]:
METRICS = [
    accuracy_score,
    f1_score,
    roc_auc_score,
]

In [5]:
results = librun.run(
    matrix_A, 
    fragment_run_start=0, 
    fragment_run_end=1, 
    metrics=METRICS
)
results_plots = [result[0] for result in results]
classifiers = [result[1] for result in results]
classifiers = [clfs for clfs in classifiers]

['C:\\Users\\Zac\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\research-KKf5lz0r-py3.8\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Zac\\AppData\\Roaming\\jupyter\\runtime\\kernel-e400aa99-1712-4e48-ba03-d4b5bf4f5f4e.json']


Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
for clfs in classifiers:
    for store in clfs:
        store.close()

In [7]:
from importlib import reload; reload(librun)

<module 'librun' from 'C:\\Users\\Zac\\Programming\\python\\research\\librun.py'>

In [21]:
results = librun.run(
    matrix_B, 
    fragment_run_start=0, 
    fragment_run_end=1, 
    metrics=METRICS,
    #force_run=True
)
results_plots = [result[0] for result in results]
classifiers = [result[1] for result in results]
classifiers = [clf for clf in classifiers]

['C:\\Users\\Zac\\AppData\\Local\\pypoetry\\Cache\\virtualenvs\\research-KKf5lz0r-py3.8\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\Zac\\AppData\\Roaming\\jupyter\\runtime\\kernel-e400aa99-1712-4e48-ba03-d4b5bf4f5f4e.json']


Experiment:   0%|          | 0/1 [00:00<?, ?it/s]

In [22]:
len(classifiers[0][0])

101

In [23]:
for name in results_plots[0][1][0].metrics:
    print(name)

AttributeError: 'DataFrame' object has no attribute 'metrics'

In [25]:
results_plots[0][1][0]

,x,accuracy_score,f1_score,roc_auc_score,time,uncertainty_average,uncertainty_min,uncertainty_max,uncertainty_variance,uncertainty_average_selected,...,uncertainty_max_selected,uncertainty_variance_selected,entropy_max,n_support,contradictory_information,expected_error,expected_error_min,expected_error_max,expected_error_average,expected_error_variance
0,10.0,0.852280,0.752080,0.848927,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,8.0,NaN,NaN,NaN,NaN,NaN,NaN
1,20.0,0.853272,0.728274,0.913717,0.280063,0.299181,1.000000e-07,0.500000,0.012994,0.410475,...,0.490156,0.004686,0.693147,13.0,6.258192,NaN,204.603526,391.837432,297.237406,497.394702
2,30.0,0.866821,0.759690,0.926770,0.334870,0.203663,3.008704e-14,0.500000,0.012135,0.405451,...,0.493287,0.002723,0.693147,15.0,6.114560,NaN,161.099549,264.492240,199.006314,84.159794
3,40.0,0.892267,0.817265,0.927998,0.393037,0.146079,3.008704e-14,0.500000,0.015509,0.445682,...,0.490807,0.001050,0.693147,23.0,6.129014,NaN,119.380909,226.900669,155.352906,228.250337
4,50.0,0.901190,0.832305,0.937219,0.458356,0.113351,3.008704e-14,0.500000,0.016517,0.397543,...,0.500000,0.005605,0.693147,31.0,2.190099,NaN,101.617985,195.620596,126.302394,145.615108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,970.0,0.968605,0.950079,0.985144,5.042113,0.038033,3.008704e-14,0.500000,0.005364,0.451808,...,0.500000,0.001023,0.693147,195.0,5.244487,NaN,31.154512,67.659058,41.630627,16.387297
97,980.0,0.968605,0.950131,0.985164,5.068008,0.047773,3.008704e-14,0.493804,0.007032,0.453753,...,0.493804,0.000725,0.693070,194.0,7.958569,NaN,33.220973,65.223486,43.609326,14.568594
98,990.0,0.968275,0.949580,0.985057,5.161939,0.038487,3.008704e-14,0.491836,0.005274,0.420695,...,0.491836,0.002039,0.693014,200.0,1.997536,NaN,32.808515,59.805962,42.633424,16.317319
99,1000.0,0.968605,0.950079,0.985031,5.977851,0.039509,3.008704e-14,0.500000,0.005773,0.454178,...,0.500000,0.000800,0.693147,201.0,8.944473,NaN,30.086264,55.237949,39.310751,13.617734
